# Implementation of a Devito skew self adjoint variable density visco- acoustic isotropic modeling operator <br>-- Correctness Testing --

## This operator is contributed by Chevron Energy Technology Company (2020)

This operator is based on simplfications of the systems presented in:
<br>**Self-adjoint, energy-conserving second-order pseudoacoustic systems for VTI and TTI media for reverse time migration and full-waveform inversion** (2016)
<br>Kenneth Bube, John Washbourne, Raymond Ergas, and Tamas Nemeth
<br>SEG Technical Program Expanded Abstracts
<br>https://library.seg.org/doi/10.1190/segam2016-13878451.1

## Introduction 

The goal of this tutorial set is to generate and prove correctness of modeling and inversion capability in Devito for variable density visco- acoustics using an energy conserving form of the wave equation. We describe how the linearization of the energy conserving *skew self adjoint* system with respect to modeling parameters allows using the same modeling system for all nonlinear and linearized forward and adjoint finite difference evolutions. 

There are three notebooks in this series:

1. *Implementation of a Devito skew self adjoint variable density visco- acoustic isotropic modeling operator -- Nonlinear Ops*
[ssa_01_iso_implementation1.ipynb](ssa_01_iso_implementation1.ipynb)
<br>Implement the nonlinear modeling operations. 

2. *Implementation of a Devito skew self adjoint variable density visco- acoustic isotropic modeling operator -- Linearized Ops*
[ssa_02_iso_implementation2.ipynb](ssa_02_iso_implementation2.ipynb)
<br>Implement the linearized (Jacobian) ```forward``` and ```adjoint``` modeling operations.

3. *Implementation of a Devito skew self adjoint variable density visco- acoustic isotropic modeling operator -- Correctness Testing*
[ssa_03_iso_correctness.ipynb](ssa_03_iso_correctness.ipynb)
<br>Tests the correctness of the implemented operators.

There are similar series of notebooks implementing and testing operators for VTI and TTI anisotropy ([README.md](README.md)).

Below we describe a suite of unit tests that prove correctness for our *skew self adjoint* operators.

## Outline 
1. Define symbols [[link]](#c_symbols) 
2. Definition of correctness tests [[link]](#c_howto) 
1. Analytic response in the far field [[link]]("#c_analytic")
2. Modeling operator linearity test, with respect to source [[link]]("#c_F_linearity")
3. Modeling operator adjoint test, with respect to source [[link]]("#c_F_adjoint")
4. Nonlinear operator linearization test, with respect to model [[link]]("#c_F_linearization")
5. Jacobian operator linearity test, with respect to model [[link]]("#c_J_linearity")
6. Jacobian operator adjoint test, with respect to model [[link]]("#c_J_adjoint")


<a id="c_symbols"></a>
## Table of symbols

Note we are only showing the symbols here relevant to the correctness tests, for more detail on notations see the implementation notebooks that precede this one.

| Symbol &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; | Description  | Dimensionality | 
|:---|:---|:---|
| $v(x,y,z)$ | Total P wave velocity ($v_0+\delta v$) | function of space |
| $v_0(x,y,z)$ | Reference P wave velocity    | function of space |
| $\delta v(x,y,z)$ | Perturbation to P wave velocity    | function of space |
| $P(t,x,y,z)$ | Total pressure wavefield ($P_0+\delta P$)| function of time and space |
| $P_0(t,x,y,z)$ | Reference pressure wavefield | function of time and space |
| $\delta P(t,x,y,z)$ | Perturbation to pressure wavefield | function of time and space |
| $s(t,x,y,z)$ | Source wavefield | function of time, localized in space to source location |
| $R(t,x,y,z)$ | Receiver wavefield | function of time, localized in space to receiver locations |
| $\widetilde{R}(t,x,y,z)$ | Receiver wavefield, time reversed | function of time, localized in space to receiver locations |
| $\Gamma,\ \Gamma^\top $ | Receiver interpolation operator, and adjoint | $R(t,x,y,z) = \Gamma\ P(t,x,y,z)$ |
| $F_v(s)$ | Forward modeling operator | Fixed $v$, Linear in $s$, maps $s(t,x,y,z)$ to $R(t,x,y,z)$ |
| $F_v^\top(R)$ | Adjoint modeling operator | Fixed $v$, Linear in $R$, maps $R(t,x,y,z)$ to $s(t,x,y,z)$ |
| $f[v]$ | Forward nonlinear modeling operator | Nonlinear in $v$, maps $v(x,y,z)$ to $R(t,x,y,z)$ |
| $J_v(\delta v) = \frac{\partial}{\partial v} f[v]$ | Forward Jacobian modeling operator | Linearized at $v$, maps $\delta v(x,y,z)$ to $\delta R(t,x,y,z)$ |
| $J_v^\top(\delta R) = \left(\frac{\partial}{\partial v} f[v]\right)^\top$ | Adjoint Jacobian modeling operator | Linearized at $v$, maps $\delta R(t,x,y,z)$ to $\delta v(x,y,z)$ |


<a id="c_howto"></a>
## Definition of correctness tests

We believe that if an operator passes the following suite of unit tests, it can be considered to be *righteous*.

1. #### Analytic response in the far field
Test that data generated in a wholespace matches analogous analytic data away from the near field. We re-use the material shown in the [examples/seismic/acoustic/accuracy.ipynb](https://github.com/devitocodes/devito/blob/master/examples/seismic/acoustic/accuracy.ipynb) notebook. 
<br>

2. #### Modeling operator linearity test, with respect to source
For random vectors $s$ and $R$, prove:
$$
\begin{aligned}
F_v(\alpha\ s) &\approx \alpha\ F_v(s) \\[5pt]
F_v^\top(\alpha\ R) &\approx \alpha\ F_v^\top(R) \\[5pt]
\end{aligned}
$$
<br>

3. #### Modeling operator adjoint test, with respect to source
For random vectors $s_1$ and $R_1$, prove:
$$
R_1 \cdot F_v(s_1) \approx s_1 \cdot F_v^\top(R_1)
$$
<br>

4. #### Nonlinear operator linearization test, with respect to model
For initial velocity $v$ and random perturbation $\delta v$ prove that the $L_2$ norm error in the linearization $E(h)$ is second order (decreases quadratically) with the magnitude of the perturbation.
$$
E = \biggl\|\ f(v+h\ \delta v) - f(v) - h\ J_v(\delta v)\ \biggr\|
$$
One way to do this is to run a suite of $h$ values decreasing by a factor of $\gamma$, and prove the error decreases by a factor of $\gamma^2$:  
$$
\frac{E\left(h\right)}{E\left(\frac{\displaystyle h}{\displaystyle \gamma}\right)} \approx \gamma^2
$$
Elsewhere in Devito tutorials, this relation is proven by fitting a line to a sequence of $E(h)$ for various $h$ and showing second order error decrease.
<br>

5. #### Jacobian operator linearity test, with respect to model
For initial velocity $v$ and random vectors $\delta v$ and $\delta R$, prove:
$$
\begin{aligned}
J_v(\alpha\ \delta v) &\approx \alpha\ J_v(\delta v) \\[5pt]
J_v^\top(\alpha\ \delta R) &\approx \alpha\ J_v^\top(\delta R)
\end{aligned}
$$
<br>

6. #### Jacobian operator adjoint test, with respect to model
For initial velocity $v$ and random vectors $\delta v_1$ and $\delta R_1$, prove:
$$
\delta R_1 \cdot J_v(\delta v_1) \approx \delta v_1 \cdot J_v^\top(\delta R_1)
$$
<br>

In addition to these tests, recall that we have already shown a unit test that demonstrates skew symmetry of the Devito generated shifted derivatives. We will include this in our suite of unit tests for completeness. Please see the first notebook for that demonstration, verifying the following relation:

$$
x_2 \cdot \left( \overrightarrow{\partial_x}\ x_1 \right) \approx -\ 
x_1 \cdot \left( \overleftarrow{\partial_x}\ x_2 \right) 
$$

## Implementation of correctness tests

Below we implement the correctness tests described above. These tests are copied from standalone tests that run in the Devito project *continuous integration* (CI) pipeline. We will implement the test methods in one cell and then call from the next cell to verify correctness, but note that a wider variety of parameterization is tested in the CI pipeline.

For these tests we use the convenience functions implemented in ```operators.py``` and ```wavesolver.py``` rather than implement the operators in the notebook as we have in the first two notebooks in this series. Please review the source to compare with our notebook implementations:
- [operators.py](operators.py)
- [wavesolver.py](wavesolver.py)

## Imports 

We have grouped all imports used in this notebook here for consistency.

In [1]:
import numpy as np
from examples.seismic import RickerSource, Receiver, TimeAxis
from devito import (Grid, Function, TimeFunction, SpaceDimension, Constant, 
                    Eq, Operator, solve, configuration)
from devito.finite_differences import Derivative
from devito.builtins import gaussian_smooth
from examples.seismic.skew_self_adjoint import *
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from timeit import default_timer as timer

# These lines force images to be displayed in the notebook, and scale up fonts 
%matplotlib inline
mpl.rc('font', size=14)

# We define 32 bit floating point as the precision type 
dtype = np.float32

# Set the default language to openmp
configuration['language'] = 'openmp'

# Set logging to debug, captures statistics on the performance of operators
configuration['log-level'] = 'DEBUG'

<a id="c_analytic"></a>
## 1. Analytic response in the far field
Test that data generated in a wholespace matches analogous analytic data away from the near field. We re-use the material shown in the [examples/seismic/acoustic/accuracy.ipynb](https://github.com/devitocodes/devito/blob/master/examples/seismic/acoustic/accuracy.ipynb) notebook. 

<a id="c_F_linearity"></a>
## 2. Modeling operator linearity test, with respect to source
For random vectors $s$ and $R$, prove:
$$
\begin{aligned}
F_v(\alpha\ s) &\approx \alpha\ F_v(s) \\[5pt]
F_v^\top(\alpha\ R) &\approx \alpha\ F_v^\top(R) \\[5pt]
\end{aligned}
$$


In [2]:
def test_ssa_iso_linearity_F(self, shape, space_order):
    """
    Linearity test for the SSA isotropic nonlinear modeling operator.
    F = modeling operator with velocity fixed
    s = source signature
    R = receiver gather
    
    This function tests the following:
        F(a s) ~ a F(s)
        F^T(a R) ~ a F^T(R)
    """
    tn = 500.  # Final time

    
    
    # Create solver from preset
    solver = setup_func(shape=shape, spacing=[15. for _ in shape], kernel=kernel, 
                        nbl=10, tn=tn, space_order=space_order, 
                        **(presets[mkey]), dtype=np.float64)

    # Create adjoint receiver symbol
    srca = Receiver(name='srca', grid=solver.model.grid,
                    time_range=solver.geometry.time_axis,
                    coordinates=solver.geometry.src_positions)

    # Run forward and adjoint operators
    rec = solver.forward(save=False)[0]
    solver.adjoint(rec=rec, srca=srca)

    # Adjoint test: Verify <Ax,y> matches  <x, A^Ty> closely
    term1 = np.dot(srca.data.reshape(-1), solver.geometry.src.data)
    term2 = norm(rec) ** 2
    info('<x, A^Ty>: %f, <Ax,y>: %f, difference: %4.4e, ratio: %f'
         % (term1, term2, (term1 - term2)/term1, term1 / term2))
    
    assert np.isclose((term1 - term2)/term1, 0., atol=1.e-11)
    

<a id="c_F_adjoint"></a>
## 3. Modeling operator adjoint test, with respect to source
For random vectors $s_1$ and $R_1$, prove:
$$
R_1 \cdot F_v(s_1) \approx s_1 \cdot F_v^\top(R_1)
$$
<br>



<a id="c_F_linearization"></a>
## 4. Nonlinear operator linearization test, with respect to model
For initial velocity $v$ and random perturbation $\delta v$ prove that the $L_2$ norm error in the linearization $E(h)$ is second order (decreases quadratically) with the magnitude of the perturbation.
$$
E = \biggl\|\ f(v+h\ \delta v) - f(v) - h\ J_v(\delta v)\ \biggr\|
$$
One way to do this is to run a suite of $h$ values decreasing by a factor of $\gamma$, and prove the error decreases by a factor of $\gamma^2$:  
$$
\frac{E\left(h\right)}{E\left(\frac{\displaystyle h}{\displaystyle \gamma}\right)} \approx \gamma^2
$$
Elsewhere in Devito tutorials, this relation is proven by fitting a line to a sequence of $E(h)$ for various $h$ and showing second order error decrease.


<a id="c_J_linearity"></a>
## 5. Jacobian operator linearity test, with respect to model
For initial velocity $v$ and random vectors $\delta v$ and $\delta R$, prove:
$$
\begin{aligned}
J_v(\alpha\ \delta v) &\approx \alpha\ J_v(\delta v) \\[5pt]
J_v^\top(\alpha\ \delta R) &\approx \alpha\ J_v^\top(\delta R)
\end{aligned}
$$


<a id="c_J_adjoint"></a>
## 6. Jacobian operator adjoint test, with respect to model
For initial velocity $v$ and random vectors $\delta v_1$ and $\delta R_1$, prove:
$$
\delta R_1 \cdot J_v(\delta v_1) \approx \delta v_1 \cdot J_v^\top(\delta R_1)
$$


<a id="nl_time_update"></a>

## Ignore this 

stripped down version of time step equation from ssa_01_implementation1 exploring the forward derivative in time for the attenuation term.

$$
\frac{\partial^2}{\partial t^2} P = \frac{\displaystyle P(t+\delta t) 
    - 2 P(t) + P(t-\delta t)}{\displaystyle \delta t^2} 
$$

<br>

$$
P(t+\delta t) = \delta t^2 \frac{\displaystyle \partial^2}{\displaystyle \partial t^2} P 
    + 2 P(t) - P(t-\delta t)
$$

<br>

$$
\frac{\partial}{\partial t} P = \frac{\displaystyle P(t+\delta t) - P(t)}{\displaystyle \delta t}
$$

<br>

$$
\begin{aligned}
\frac{b}{v^2} \left( \frac{\omega}{Q} \frac{\partial}{\partial t} P +
    \frac{\partial^2}{\partial t^2} P \right) &=
    \overleftarrow{\partial x}\left(b\ \overrightarrow{\partial x} P \right) +
    \overleftarrow{\partial y}\left(b\ \overrightarrow{\partial y} P \right) +
    \overleftarrow{\partial z}\left(b\ \overrightarrow{\partial z} P \right) + s\\[10pt]
    \frac{\partial^2}{\partial t^2} P &=
    \frac{v^2}{b} \left[ 
        \overleftarrow{\partial x}\left(b\ \overrightarrow{\partial x} P \right) +
        \overleftarrow{\partial y}\left(b\ \overrightarrow{\partial y} P \right) +
        \overleftarrow{\partial z}\left(b\ \overrightarrow{\partial z} P \right) + s
    \right] 
    - \frac{\omega}{Q} \frac{\partial}{\partial t} P
\end{aligned}
$$

<br>

$$
P(t+\delta t) = - \delta t^2 \frac{\omega}{Q}
    \left( \frac{\displaystyle P(t+\delta t) - P(t)}{\displaystyle \delta t} \right) 
        + 2 P(t) - P(t-\delta t)
$$

<br>

$$
\begin{aligned}
    P(t+\delta t) &= \delta t^2 
        \frac{v^2}{b} \left[ 
            \overleftarrow{\partial x}\left(b\ \overrightarrow{\partial x} P \right) +
            \overleftarrow{\partial y}\left(b\ \overrightarrow{\partial y} P \right) +
            \overleftarrow{\partial z}\left(b\ \overrightarrow{\partial z} P \right) + s
        \right] \\[10pt]
        & \quad 
            - \delta t \frac{\omega}{Q} P(t+\delta t) + \delta t \frac{\omega}{Q} P(t)
            + 2 P(t) - P(t-\delta t) \\[20pt]
    \left(1 + \delta t \frac{\omega}{Q} \right) P(t+\delta t) &= \delta t^2
            \frac{v^2}{b} \left[ 
                \overleftarrow{\partial x}\left(b\ \overrightarrow{\partial x} P \right) +
                \overleftarrow{\partial y}\left(b\ \overrightarrow{\partial y} P \right) +
                \overleftarrow{\partial z}\left(b\ \overrightarrow{\partial z} P \right) + s
            \right] \\[10pt]
        & \quad 
            + \left(2 + \delta t \frac{\omega}{Q} \right)P(t) - P(t-\delta t) 
\end{aligned}
$$

<br>

$$
P(t+\delta t) = \left(1 + \delta t \frac{\omega}{Q} \right)^{-1}
    \left\{ \delta t^2
        \frac{v^2}{b} \left[ 
            \overleftarrow{\partial x}\left(b\ \overrightarrow{\partial x} P \right) +
            \overleftarrow{\partial y}\left(b\ \overrightarrow{\partial y} P \right) +
            \overleftarrow{\partial z}\left(b\ \overrightarrow{\partial z} P \right) + s
        \right] + \left(2 + \delta t \frac{\omega}{Q} \right)P(t) - P(t-\delta t) \right\}
$$
